In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import pandas as pd
import altair
import seaborn
import geopandas
import geoplot
RANDOM_SEED = 511
rng = np.random.default_rng(RANDOM_SEED)
seaborn.set_theme(style="darkgrid")
altair.renderers.enable('mimetype')
db_url = os.getenv('DB_URL')
%load_ext autoreload
%autoreload 2

In [ ]:
sql = """
    with transformed as (
        SELECT 
            zipcode,
            ST_Transform(ST_SetSRID(wkb_geometry, 102718), 4326) :: geography AS boundary
        FROM zip_code_geometries
    )
    select
        *,
        ST_Centroid(boundary) AS centroid
    from transformed
"""
import shapely.wkb
zip_boundaries = geopandas.GeoDataFrame.from_postgis(
    sql, db_url, 
    geom_col='boundary', index_col='zipcode')

zip_boundaries['centroid'] = zip_boundaries['centroid'].apply(shapely.wkb.loads)
print(zip_boundaries.shape)
zip_boundaries.head(3).T

In [ ]:
zip_centroids = zip_boundaries.set_geometry('centroid')

In [ ]:
ax = zip_boundaries.boundary.plot(figsize=(12,8))
zip_centroids.centroid.plot(ax=ax, markersize=0.5)


In [ ]:
sql = """
    select
        to_zip_code as zip_code,
        travel_time / 60 as travel_time,
        z.boundary
    from zip_code_distances
        left join zip_codes z on to_zip_code = z.zip_code
    where from_zip_code = '10037'
"""
zip_travel_times = geopandas.GeoDataFrame.from_postgis(
    sql, db_url, 
    geom_col='boundary', index_col='zip_code')

zip_travel_times.head(3).T

In [ ]:
zip_travel_times.explore(
    column='travel_time'
)